In [ ]:
%install '.package(path: "$cwd/FastaiNotebook_04_callbacks")' FastaiNotebook_04_callbacks

In [ ]:
import FastaiNotebook_04_callbacks
import TensorFlow
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")
let plt = Python.import("matplotlib.pyplot")

# A CNN Mnist Model

In [ ]:
let data = mnistDataBunch(flat: false, bs: 512)

In [ ]:
let firstBatch = data.train.first(where: { _ in true })!
let batchShape = firstBatch.xb.shape
let batchSize = batchShape.dimensions[0]
let exampleSideSize = batchShape.dimensions[1]
assert(exampleSideSize == batchShape.dimensions[2])
print("Batch size: \(batchSize)")
print("Example side size: \(exampleSideSize)")

let classCount = firstBatch.yb.shape.dimensions[1]
print("Class count: \(classCount)")

In [ ]:
struct CnnModel: Layer {
    var reshapeToSquare = Reshape<Float>([-1, exampleSideSize, exampleSideSize, 1])
    var conv1 = Conv2D<Float>(
        filterShape: (5, 5, 1, 8),
        strides: (2, 2),
        padding: .same,
        activation: relu)
    var conv2 = Conv2D<Float>(
        filterShape: (3, 3, 8, 16),
        strides: (2, 2),
        padding: .same,
        activation: relu)
    var conv3 = Conv2D<Float>(
        filterShape: (3, 3, 16, 32),
        strides: (2, 2),
        padding: .same,
        activation: relu)
    var conv4 = Conv2D<Float>(
        filterShape: (3, 3, 32, 32),
        strides: (2, 2),
        padding: .same,
        activation: relu)
    
    // The Python notebook uses "AdaptiveAvgPool2d", which I assume is different from "AvgPool2D".
    // But our layers lib only has "AvgPool2D" and that sounds good enough for now.
    var pool = AvgPool2D<Float>(poolSize: (2, 2), strides: (1, 1))
    
    var flatten = Flatten<Float>()
    var linear = Dense<Float>(inputSize: 32, outputSize: Int(classCount))
    
    @differentiable
    func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        // There isn't a "sequenced" defined with enough layers.
        let intermediate =  input.sequenced(
            in: context,
            through: reshapeToSquare, conv1, conv2, conv3, conv4)
        return intermediate.sequenced(in: context, through: pool, flatten, linear)
    }
}

In [ ]:
// Test that data goes through the model as expected.
let predictions = CnnModel().applied(to: firstBatch.xb, in: Context(learningPhase: .training))
print(predictions.shape)
print(predictions[0])

# Compare training on CPU and GPU

In [ ]:
let opt = SGD<CnnModel, Float>(learningRate: 0.4)
func modelInit() -> CnnModel { return CnnModel() }

// TODO: When TF-421 is fixed, switch back to the normal `softmaxCrossEntropy`.

@differentiable(vjp: _vjpSoftmaxCrossEntropy)
func softmaxCrossEntropy1<Scalar: TensorFlowFloatingPoint>(
    _ features: Tensor<Scalar>, _ labels: Tensor<Scalar>
) -> Tensor<Scalar> {
    return Raw.softmaxCrossEntropyWithLogits(features: features, labels: labels).loss.mean()
}

@usableFromInline
func _vjpSoftmaxCrossEntropy<Scalar: TensorFlowFloatingPoint>(
    features: Tensor<Scalar>, labels: Tensor<Scalar>
) -> (Tensor<Scalar>, (Tensor<Scalar>) -> (Tensor<Scalar>, Tensor<Scalar>)) {
    let (loss, grad) = Raw.softmaxCrossEntropyWithLogits(features: features, labels: labels)
    let batchSize = Tensor<Scalar>(features.shapeTensor[0])
    return (loss.mean(), { v in ((v / batchSize) * grad, Tensor<Scalar>(0)) })
}

let learner = Learner(data: data, lossFunction: softmaxCrossEntropy1, optimizer: opt, initializingWith: modelInit)
learner.delegates = [Learner.TrainEvalDelegate(), Learner.AvgMetric(metrics: [accuracy])]

In [ ]:
// This happens on the GPU (if you have one and it's configured correctly).
// I tried this on a GCE 8vCPU 30GB + Tesla P100:
// - time: ~4.3s
// - nvidia-smi shows ~10% GPU-Util while this is running
time {
    try! learner.fit(1)
}

In [ ]:
// This happens on the CPU.
// I tried this on a GCE 8vCPU 30GB + Tesla P100:
// - time: ~6.3s
// - nvidia-smi shows 0% GPU-Util while this is running
time {
    withDevice(.cpu) {
        try! learner.fit(1)
    }
}

# Collect Layer Activation Statistics

In [ ]:
class LayerDelegate {
    func layer(_ name: String, didProduceActivation activation: Tensor<Float>, in context: Context) {}
}

// TODO: This should not be a global. Maybe we can put it in Context?
var layerDelegates: [LayerDelegate] = []

struct DelegatingLayer<L: Layer>: Layer
    where L.Output == Tensor<Float>,
          L.CotangentVector == L.AllDifferentiableVariables
{
    @noDerivative let name: String
    var layer: L
    
    init(_ name: String, _ layer: L) {
        self.name = name
        self.layer = layer
    }
    
    @differentiable
    func applied(to input: L.Input, in context: Context) -> L.Output {
        let activation = layer.applied(to: input, in: context)
        
        // Need to wrap this in a closure to prevent AD from seeing control flow.
        ({
            layerDelegates.forEach { $0.layer(name, didProduceActivation: activation, in: context) }
        })()
        
        return activation
    }
}

In [ ]:
struct CnnModelWithDelegates: Layer {
    var reshape = Reshape<Float>([-1, exampleSideSize, exampleSideSize, 1])

    var conv1 = DelegatingLayer(
        "conv1",
        Conv2D<Float>(filterShape: (5, 5, 1, 8), strides: (2, 2), padding: .same, activation: relu))
    
    var conv2 = DelegatingLayer(
        "conv2",
        Conv2D<Float>(filterShape: (3, 3, 8, 16), strides: (2, 2), padding: .same, activation: relu))
        
    var conv3 = DelegatingLayer(
        "conv3",
        Conv2D<Float>(filterShape: (3, 3, 16, 32), strides: (2, 2), padding: .same, activation: relu))
        
    var conv4 = DelegatingLayer(
        "conv4",
        Conv2D<Float>(filterShape: (3, 3, 32, 32), strides: (2, 2), padding: .same, activation: relu))
    
    // The Python notebook uses "AdaptiveAvgPool2d", which I assume is different from "AvgPool2D".
    // But our layers lib only has "AvgPool2D" and that sounds good enough for now.
    var pool = DelegatingLayer(
        "pool",
        AvgPool2D<Float>(poolSize: (2, 2), strides: (1, 1)))
    
    var flatten = Flatten<Float>()
        
    var linear = DelegatingLayer(
        "linear",
        Dense<Float>(inputSize: 32, outputSize: Int(classCount)))
    
    @differentiable
    func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        // There isn't a "sequenced" defined with enough layers.
        let intermediate =  input.sequenced(
            in: context,
            through: reshape, conv1, conv2, conv3, conv4)
        return intermediate.sequenced(in: context, through: pool, flatten, linear)
    }
}

In [ ]:
class ActivationStatistics: LayerDelegate {
    var activationMeans: [String: [Float]] = [:]
    var activationStds: [String: [Float]] = [:]    
    override func layer(_ name: String, didProduceActivation activation: Tensor<Float>, in context: Context) {
        guard context.learningPhase == .training else { return }
        activationMeans[name, default: []].append(activation.mean().scalar!)
        activationStds[name, default: []].append(activation.standardDeviation().reshaped(to: []).scalar!)
    }
}
layerDelegates.append(ActivationStatistics())

In [ ]:
let opt = SGD<CnnModelWithDelegates, Float>(learningRate: 0.4)
func modelInit() -> CnnModelWithDelegates { return CnnModelWithDelegates() }
let learner = Learner(data: data, lossFunction: softmaxCrossEntropy1, optimizer: opt, initializingWith: modelInit)
learner.delegates = [Learner.TrainEvalDelegate(), Learner.AvgMetric(metrics: [accuracy])]

In [ ]:
// This LayerDelegate stuff slows it down to ~6s/epoch.
time {
    try! learner.fit(2)
}

In [ ]:
let activationStatistics = layerDelegates[0] as! ActivationStatistics
var names: [String] = []
for name in activationStatistics.activationMeans.keys.sorted() {
    plt.plot(activationStatistics.activationMeans[name])
    names.append(name)
}
plt.legend(names)
plt.show()

In [ ]:
var names: [String] = []
for name in activationStatistics.activationStds.keys.sorted() {
    plt.plot(activationStatistics.activationStds[name])
    names.append(name)
}
plt.legend(names)
plt.show()